In [22]:
import basedosdados as bd
import pandas as pd
import folium
import plotly.express as px
from matplotlib import pyplot
from shapely.geometry.polygon import Polygon
import geopandas as gpd

In [51]:
# ainda há um problema com duplicação dos shapes durante a materialização que está sendo investigado
shapes = """
with shape as (
  select 
    *,
    row_number() over(
      partition by trip_id, shape_id, route_id
    ) rn,
    ST_ASGEOJSON(ST_BUFFER(shape, 50)) buffer_50m,
    ST_ASGEOJSON(ST_BUFFER(shape, 100)) buffer_100m,
    ST_ASGEOJSON(ST_BUFFER(shape, 150)) buffer_150m,
    ST_ASGEOJSON(ST_BUFFER(shape, 200)) buffer_200m
  from `rj-smtr-dev.br_rj_riodejaneiro_sigmob.shapes_geom`
  where data_versao = (select max(data_versao) from `rj-smtr-dev.br_rj_riodejaneiro_sigmob.shapes_geom`)
  and linha_gtfs = '371'
)
select * except(rn)
from shape
where rn=1
"""

In [52]:
registros = """
select 
  * except(versao)
from `rj-smtr.br_rj_riodejaneiro_veiculos.gps_sppo`
where data between '2022-03-28' and '2022-04-04' 
and servico = '371'
and extract(hour from timestamp_gps) between 7 and 8
"""

In [37]:
df = pd.read_csv('sppo_servico=371_hora_between_7_and_8.csv')
df.sort_values(by=['id_veiculo','timestamp_gps'], inplace=True, ignore_index=True)
df

,modo,timestamp_gps,data,hora,id_veiculo,servico,latitude,longitude,flag_em_operacao,flag_em_movimento,tipo_parada,flag_linha_existe_sigmob,flag_trajeto_correto,flag_trajeto_correto_hist,status,velocidade_instantanea,velocidade_estimada_10_min,distancia
0,SPPO,2022-03-28 07:00:47,2022-03-28,07:00:47,C51502,371,-22.87791,-43.33603,True,True,terminal,True,True,True,Em operação,0,14.3,100.220958
1,SPPO,2022-03-28 07:01:47,2022-03-28,07:01:47,C51502,371,-22.87771,-43.33634,True,True,terminal,True,True,True,Em operação,18,14.2,38.771143
2,SPPO,2022-03-28 07:02:47,2022-03-28,07:02:47,C51502,371,-22.87609,-43.33913,True,True,terminal,True,True,True,Em operação,22,14.9,337.859329
3,SPPO,2022-03-28 07:03:47,2022-03-28,07:03:47,C51502,371,-22.87681,-43.33984,True,True,terminal,True,True,True,Em operação,0,13.1,108.169400
4,SPPO,2022-03-28 07:04:47,2022-03-28,07:04:47,C51502,371,-22.87710,-43.34012,True,True,terminal,True,True,True,Em operação,3,10.3,43.159096
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26861,SPPO,2022-04-04 08:55:49,2022-04-04,08:55:49,C51616,371,-22.88253,-43.27596,True,True,NaN,True,True,True,Em operação,31,18.3,69.204447
26862,SPPO,2022-04-04 08:57:19,2022-04-04,08:57:19,C51616,371,-22.88552,-43.27877,True,True,NaN,True,True,True,Em operação,41,17.9,439.778672
26863,SPPO,2022-04-04 08:57:49,2022-04-04,08:57:49,C51616,371,-22.88734,-43.28193,True,True,NaN,True,True,True,Em operação,35,20.4,381.768605
26864,SPPO,2022-04-04 08:58:49,2022-04-04,08:58:49,C51616,371,-22.88773,-43.28395,True,True,NaN,True,True,True,Em operação,6,19.9,211.425505


In [38]:
buffers = pd.read_csv("371_shapes_buffers.csv")

In [48]:
m = folium.Map(max_zoom=100, zoom_start=100)
# for row in buffers:
#     print(row)

In [49]:
for i, row in buffers.iterrows():
    folium.GeoJson(row['buffer_50m'], tooltip = row['linha_gtfs']).add_to(m)

In [50]:
for i,row in df[df['id_veiculo']=='C51502'].iterrows():
    folium.CircleMarker(location=(row['latitude'],row['longitude'])).add_to(m)
m